Copyright **`(c)`** 2023 Florentin-Cristian Udrea

In [86]:
from itertools import product
from random import random, randint, shuffle, seed
import numpy as np
from scipy import sparse
from copy import deepcopy
from functools import reduce
from collections import namedtuple, deque
from queue import PriorityQueue
import tqdm

In [87]:
def make_set_covering_problem(num_points, num_sets, density):
    """Returns a sparse array where rows are sets and columns are the covered items"""
    seed(num_points*2654435761+num_sets+density)
    sets = sparse.lil_array((num_sets, num_points), dtype=bool)
    for s, p in product(range(num_sets), range(num_points)):
        if random() < density:
            sets[s, p] = True
    for p in range(num_points):
        sets[randint(0, num_sets-1), p] = True
    return sets

# Halloween Challenge

Find the best solution with the fewest calls to the fitness functions for:

* `num_points = [100, 1_000, 5_000]`
* `num_sets = num_points`
* `density = [.3, .7]` 

Best solutions

|   | .3  | .7  | 
|---|---|---|
| 100  | 6 | 3 | 
| 1000  | 12  | 4 |
|  5000 | 17  | 6 |

Average number of calls to fitness function per solution:

|   | .3  | .7  |  
|---|---|---|
| 100  | 37.23 | 10.55 | 
| 1000  | 48.82  | 13.53 |
|  5000 | 64.26  | 18.00 | 

Num resets:

|   | .3  | .7  |  
|---|---|---|
| 100  | 1000 | 1000 | 
| 1000  | 100 | 100 |
|  5000 | 30  | 30 |  

In [88]:
NUM_POINTS = 100
PROBLEM_SIZE = 100
DENSITY = 0.3
SETS = make_set_covering_problem(NUM_POINTS, PROBLEM_SIZE, DENSITY)
print("Element at row=42 and column=42:", SETS[42, 42])

Element at row=42 and column=42: True


In [89]:
global CALLS_FITNESS
CALLS_FITNESS = 0

def fitness(state):
    global CALLS_FITNESS 
    CALLS_FITNESS += 1
    
    cost = sum(state)
    elem_set = set()
    non_zeros = SETS.nonzero() # sets, elements
    
    # for each set in solution add it's elements indexes
    # only appear once because adding to set, so it's basically OR
    valid = [elem_set.add( non_zeros[1][i] ) \
             for i in range(len(non_zeros[0])) \
             if state[ non_zeros[0][i] ] ]
    valid = len(elem_set)
    return valid, -cost


In [90]:
def tweak(state, change_meter):
    new_state = deepcopy(state)

    for _ in range(change_meter):
        index = randint(0, PROBLEM_SIZE - 1)
        new_state[index] = not new_state[index]
    
    return new_state

def tweak2(state, current_valid):
    new_state = deepcopy(state)
    
    index_ttf = np.random.choice([i for i in range(len(state)) if state[i]])
    new_state[index_ttf] = not new_state[index_ttf]
    if not current_valid:
        index_ftt = np.random.choice([i for i in range(len(state)) if not state[i]])
        new_state[index_ftt] = not new_state[index_ftt]

    return new_state

def tweak_large_random(state, current_valid, change_meter):
    new_state = deepcopy(state)
    
    index_ttf = np.random.choice([i for i in range(len(state)) if state[i]], change_meter, replace=False).tolist()
    index_ftt = []
    if not current_valid:
        index_ftt = np.random.choice([i for i in range(len(state)) if not state[i]], change_meter, replace=False).tolist()

    for i in index_ftt+index_ttf:
        new_state[i] = not new_state[i]

    return new_state

def tweak_list(state, current_fitness, seen_solutions):
    new_state = deepcopy(state)
    actions = np.array([i for i in range(len(state)) if state[i]])
    np.random.shuffle(actions)
    
    for k in actions:
        new_state[k] = not new_state[k]
        if new_state in seen_solutions:
            new_state[k] = not new_state[k]
            continue

        new_fitness = fitness(new_state)
        if new_fitness > current_fitness:
            seen_solutions.append(new_state)
            return new_state
        else:
            new_state[k] = not new_state[k] ### undo action 
        

    
    return state


In [94]:
CALLS_FITNESS = 0
NUM_RESET = 10_000
sols = []
fits = []
calls = [0]

list_of_seen_solutions = []

for reset in tqdm.tqdm(range(NUM_RESET)):    
    current_state = [np.random.choice([False]) for _ in range(PROBLEM_SIZE)]
    current_fitness = (-1, -PROBLEM_SIZE) #fitness(current_state)
    while current_fitness[0] != PROBLEM_SIZE:
        for _ in range( max(1,PROBLEM_SIZE//50) ):
            index = np.random.randint(PROBLEM_SIZE)
            current_state[index] = True

        current_fitness = fitness(current_state)
        
    if NUM_RESET == 1:
        print(f"STEP {'-':>5} | ", current_fitness)

    change_meter = -current_fitness[1] // 2 + 1

    NUM_STEPS = 1_000

    greedy=False
    for step in range(NUM_STEPS):
        change_meter = max(1, change_meter - 1)
        if change_meter == 1:
            greedy = True
            
        if greedy:
            new_state = tweak_list(current_state, current_fitness, list_of_seen_solutions)
            if new_state == current_state:
                break
        else:
            new_state = tweak_large_random(current_state, 
                            current_valid=(current_fitness[0] == PROBLEM_SIZE), 
                            change_meter=change_meter)
                        
        new_fitness = fitness(new_state)
        
        if new_fitness > current_fitness:
            current_state = new_state
            current_fitness = new_fitness

            change_meter = -current_fitness[1] // 2 
            # greedy = False  
            if NUM_RESET == 1:     
               print(f"STEP {step:5} | ", current_fitness, "(First improvement HC)" if greedy else "")

    if NUM_RESET==1:
        print(f"Calls to fitness: {CALLS_FITNESS}")
        print("Best fitness: ", current_fitness)
    sols.append(current_state)
    fits.append(current_fitness)
    calls.append(deepcopy(CALLS_FITNESS))

print("Best solution had ", end="")
print(abs(np.max([fits[i][1] for i in range(len(fits))])), end=" ")
print("sets")
print("Average calls: ", np.mean([calls[i]- calls[i-1] for i in range(1,len(calls))] ))
print("Num of saved states: ", len(list_of_seen_solutions))

100%|██████████| 10000/10000 [2:04:44<00:00,  1.34it/s] 

Best solution had 6 sets
Average calls:  37.3019
Num of saved states:  32115
